<div style="padding: 10px; margin-bottom: 10px; border: 1px solid #333333;">
    
# support mission [モデルによる撮影画像の判定]
    
- このsupport mission では、先ほど作成したmodelに基づき、カメラに映っている状況がfreeである確率とblockedである確率を返してくれる事を確認する



## 完了の定義：
- blockedの確率が表示されていること

</div>



## <span style="color: red; ">NEW</span> タグ：[モデルによる撮影画像の判定]

 
訓練されたモデルを読み込もう

トレーニングノートの指示に従って、best_model.pthをこのファイルと同じディレクトリに置いていることを前提としています。 


In [1]:
import torch
import torchvision

# 学習済みモデルを初期化しています
C_MODEL = torchvision.models.alexnet(pretrained=False)
C_MODEL.classifier[6] = torch.nn.Linear(C_MODEL.classifier[6].in_features, 2)

次に ``best_model.pth`` ファイルを読み込みます

In [2]:
C_MODEL.load_state_dict(torch.load('best_model.pth'))

現在、モデルはCPUメモリにあり、以下のコードを実行してGPUデバイスに転送します。

In [3]:
device = torch.device('cuda')
C_MODEL = C_MODEL.to(device)

## 前処理を行う関数を定義する
モデルを読み込みましたが、少し問題があります。 モデルをトレーニングした形式は、カメラの形式と完全には一致しません。 そのためには、いくつかの前処理を行う必要があり以下の手順をふむ必要があります

1. BGRからRGBに変換する
2. HWCレイアウトからCHWレイアウトに変換する
3. トレーニング中に行ったのと同じパラメーターを使用して正規化します（カメラは[0、255]の範囲の値を提供し、ロードされた画像は[0、1]の範囲でトレーニングします。したがって、255.0
4. CPUメモリからGPUメモリにデータを転送します
5. バッチディメンションを追加する

In [4]:
import cv2
import numpy as np

mean = 255.0 * np.array([0.485, 0.456, 0.406])
stdev = 255.0 * np.array([0.229, 0.224, 0.225])

normalize = torchvision.transforms.Normalize(mean, stdev)

# カメラからの画像を正規化するメソッドを定義しています
def preprocess(camera_value):
    global device, normalize
    x = camera_value
    x = cv2.cvtColor(x, cv2.COLOR_BGR2RGB)
    x = x.transpose((2, 0, 1))
    x = torch.from_numpy(x).float()
    x = normalize(x)
    x = x.to(device)
    x = x[None, ...]
    return x

これで、カメラの形式からニューラルネットワークの入力形式に画像を変換できる前処理関数を定義しました。

それでは、カメラを起動して表示しましょう。 
jetbotがblockedであると判断する確率を表示するスライダーを作成します

 ### タグ：[画像表示]をコピペしてください

In [5]:
import traitlets
import ipywidgets.widgets as widgets
from jetbot import Camera, bgr8_to_jpeg

# カメラからの映像の表示サイズを設定する
C_IMAGE = widgets.Image(format='jpeg', width=224, height=224)

#撮影される画像サイズの設定
C_CAMERA = Camera.instance(width=224, height=224)
C_CAMERA_LINK = traitlets.dlink((C_CAMERA, 'value'), (C_IMAGE, 'value'), transform=bgr8_to_jpeg)

#カメラからの映像を表示
display(C_IMAGE)

Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x02\x01\x0…

次に、カメラの値が変更されるたびに呼び出される関数を作成します。 この関数は次の手順を実行します

1. カメラ画像の前処理
2. ニューラルネットワークを実行する
3. 出力に応じて動作を変更する

In [6]:
import torch.nn.functional as F
import time

def update(change):
    global C_ROBOT
    x = change['new'] 
    x = preprocess(x)
    y = C_MODEL(x)
    
    
    y = F.softmax(y, dim=1)
    # 下記のプログラムでblockedの確率を取得している
    # datasetディレクトリを見てもらうと上がblocked,下がfreeになっている
    # つまり、blockedの確率を取得したい場合[0]番目にその値が入っており、freeの確率を持ってきたい場合[1]番目を指定する必要がある
    # blockedよりも上にディレクトリが作成された場合は、そのディレクトリの確率が[0]に入っていることになる
    # 確率prob_blockedは 0 < prob_blocked <= 1 である

    prob_blocked = float(y.flatten()[0])
    
    # 使いまわすときは下記のプログラム time.sleep(1.0),print(prob_blocked)は削除してください
    time.sleep(1.0)
    print(prob_blocked)
    ####################################################################
    time.sleep(0.001)
    
update({'new': C_CAMERA.value}) 

0.44619250297546387


In [7]:
C_CAMERA.observe(update, names='value') 

確率が表示されていることを確認してください

確率はディレクトリの順番に依存します。

例：[0.4 , 0.6]と表示された場合、datasetディレクトリの中を見てほしいのだがblockedその下にfreeという順番でディレクトリが作成されている。

つまりこの場合、0.4(40%)はblockedの確率を表し、0.6はfreeの確率を表している。